# <center>AGC Introductory Demo</center>


This demo includes a graphical interface to specify the module's properties and is meant as an introduction to it's functionality. The **Reference power**, **Max gain**, **Detector gain** and **Averaging length** fields, translate directly to the equivalent module properties.

The **Number of samples** field defines the length of the input signal. it populates a variable *n* that can be used in the **Signal** field. Any valid Python expression can be used for the input signal (defined by the *sig* variable).

The output shows plots of the input and output signals where the output power adjustment can be verified. The output power (of the last half of the output samples, to avoid the initial transients) will be numerically reported to compare against the desired value. Finally a plot of the error, as reported by the AGC, is shown. This is useful to analyse the damping factor, initial oscillation amplitude and steady-state error, for different combinations of input parameters.

A new instance should be created (using the **Init** button), whenever new parameters (or a new unrelated input signal) are specified. To clear the ouput, use the Jupyter Notebook's own shortcuts. Refer to the documentation in the Help menu.

In [1]:
%matplotlib inline

import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np

import sksdr
import utils

agc = None
output_samples = None

def init(b):
    global agc, disp
    ref_power = ref_power_widget.value
    max_gain = max_gain_widget.value
    det_gain = det_gain_widget.value
    avg_len = avg_len_widget.value
    agc = sksdr.AGC(ref_power, max_gain, det_gain, avg_len)
    with disp:
        print('Initiated with ' + repr(agc))

def execute(b):
    global agc, output_samples, disp
    n = np.arange(num_samples_widget.value)
    _locals = {'n': n}
    exec(signal_widget.value, None, _locals)
    sig = _locals['sig']
    fig = plt.figure(figsize=(15,10))
    gs = gridspec.GridSpec(2, 2, figure=fig)
    with disp:
        sksdr.time_plot([sig], [''], [1], 'Input Signal', fig=fig, gs=gs[0, 0])
        out, err = agc(sig)
        sksdr.time_plot([out], [''], [1], 'Output Signal', fig=fig, gs=gs[0, 1])
        print('Output power: ' + str(sksdr.utils.power(out, 1 / 2)))
        sksdr.time_plot([err], [''], [1], 'Error', fig=fig, gs=gs[1, 0])
        plt.show()
        output_samples = out
        print('Output samples available in variable "output_samples"')

style = dict(utils.description_width_style)
settings_grid = widgets.GridspecLayout(3, 3)
settings_grid[0, 0] = ref_power_widget = widgets.BoundedFloatText(description='Reference power:', value=1.0, min=0.0, max=np.finfo(float).max, continuous_update=False, style=style)
settings_grid[0, 1] = max_gain_widget = widgets.BoundedFloatText(description='Max gain (dB):', value=60.0, min=0.0, max=np.finfo(float).max, continuous_update=False, style=style)
settings_grid[0, 2] = det_gain_widget = widgets.BoundedFloatText(description='Detector gain:', value=0.01, min=0.01, max=np.finfo(float).max, continuous_update=False, style=style)
settings_grid[1, 0] = avg_len_widget = widgets.BoundedIntText(description='Averaging length (samples):', value=100, min=1, max=np.iinfo(int).max, continuous_update=False, style=style)
settings_grid[2, 0] = num_samples_widget = widgets.BoundedIntText(description='Number of samples (n=):', value=1000, min=1, max=np.iinfo(int).max, continuous_update=False, style=style)
settings_grid[2, 1:] = signal_widget = widgets.Textarea(description='Signal (sig=):', value='sig = np.cos(2 * np.pi * n / 8)', continuous_update=False, style=style, layout=widgets.Layout(height='auto', width='auto'))
init_button = widgets.Button(description='Init', tooltip='Init')
init_button.on_click(init)
execute_button = widgets.Button(description='Execute', tooltip='Execute')
execute_button.on_click(execute)
disp = widgets.Output()
ui = widgets.VBox([
    settings_grid,
    widgets.HBox([init_button, execute_button]),
    disp
])
display(ui)